In [1]:
# Translated to .py by Meritxell Pacheco (December 2016)
########################################################
# Updated by Evanthia Kazagli (January 2017)
########################################################
# Adapted for PandasBiogeme by Michel Bierlaire
# Fri Nov  2 15:57:48 2018
########################################################

import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme.models import logit
from datetime import datetime

pandas = pd.read_table("swissmetro.dat")
database = db.Database("swissmetro",pandas)
pd.options.display.float_format = '{:.3g}'.format

globals().update(database.variables)
from biogeme.expressions import *

exclude = ((  PURPOSE   !=  1  ) * (  PURPOSE   !=  3  ) + (  CHOICE   ==  0  ))>0
database.remove(exclude)

#Parameters to be estimated
# Arguments:
#   1  Name for report. Typically, the same as the variable
#   2  Starting value
#   3  Lower bound
#   4  Upper bound
#   5  0: estimate the parameter, 1: keep it fixed
ASC_CAR = Beta('ASC_CAR',-1.12,None,None,0)
ASC_SBB = Beta('ASC_SBB',0,None,None,1)
ASC_SM = Beta('ASC_SM',0.665,None,None,0)
BETA_COST = Beta('BETA_COST',-3.56,None,None,0)
BETA_HE = Beta('BETA_HE',-1.14,None,None,0)
BETA_TIME = Beta('BETA_TIME',-2.90,None,None,0)
CLASSIC_mean = Beta('CLASSIC',0,None,None,1)
CLASSIC_std = Beta('CLASSIC_std',4.48,None,None,0)
RAIL_mean = Beta('RAIL',0,None,None,1)
RAIL_std = Beta('RAIL_std',4.66,None,None,0)
# Random parameters
CLASSIC_random = CLASSIC_mean + CLASSIC_std * bioDraws('CLASSIC_random','NORMAL')
RAIL_random = RAIL_mean + RAIL_std * bioDraws('RAIL_random','NORMAL')

# Define here arithmetic expressions for variables that are not directly
# available in the data file
CAR_AV_SP = DefineVariable('CAR_AV_SP', CAR_AV    *  (  SP   !=  0  ),database)
SM_COST = DefineVariable('SM_COST', SM_CO   * (  GA   ==  0  ),database)
TRAIN_AV_SP = DefineVariable('TRAIN_AV_SP', TRAIN_AV    *  (  SP   !=  0  ),database)
TRAIN_COST = DefineVariable('TRAIN_COST', TRAIN_CO   * (  GA   ==  0  ),database)
TRAIN_TT_SCALED = DefineVariable('TRAIN_TT_SCALED',\
                                 TRAIN_TT / 100.0,database)
TRAIN_COST_SCALED = DefineVariable('TRAIN_COST_SCALED',\
                                   TRAIN_COST / 100,database)
SM_TT_SCALED = DefineVariable('SM_TT_SCALED', SM_TT / 100.0,database)
SM_COST_SCALED = DefineVariable('SM_COST_SCALED', SM_COST / 100,database)
CAR_TT_SCALED = DefineVariable('CAR_TT_SCALED', CAR_TT / 100,database)
CAR_CO_SCALED = DefineVariable('CAR_CO_SCALED', CAR_CO / 100,database)
TRAIN_HE_SCALED = DefineVariable('TRAIN_HE_SCALED', TRAIN_HE / 100,database)
SM_HE_SCALED = DefineVariable('SM_HE_SCALED', SM_HE / 100,database)


# Utilities
V_Car_SP = ASC_CAR + BETA_TIME * CAR_TT_SCALED + BETA_COST * CAR_CO_SCALED + CLASSIC_random
V_SBB_SP = ASC_SBB + BETA_TIME * TRAIN_TT_SCALED + BETA_COST * TRAIN_COST_SCALED + BETA_HE * TRAIN_HE_SCALED + RAIL_random + CLASSIC_random
V_SM_SP = ASC_SM + BETA_TIME * SM_TT_SCALED + BETA_COST * SM_COST_SCALED + BETA_HE * SM_HE_SCALED + RAIL_random
#
V = {3: V_Car_SP,1: V_SBB_SP,2: V_SM_SP}
av = {3: CAR_AV_SP,1: TRAIN_AV_SP,2: SM_AV}

# Choice model
prob = logit(V,av,CHOICE)
logprob = log(MonteCarlo(prob))

biogeme = bio.BIOGEME(database,logprob,numberOfDraws=2000)
biogeme.modelName = "Mixture_SM_ErrorComp_02"
start_time = datetime.now()
results = biogeme.estimate()
print(f"Estimation time: {datetime.now() - start_time}")
# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)
print(f"Nbr of observations: {database.getNumberOfObservations()}")
print(f"LL(0) =    {results.data.initLogLike:.3f}")
print(f"LL(beta) = {results.data.logLike:.3f}")
print(f"Output file: {results.data.htmlFileName}")


/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  from ipykernel import kernelapp as app


Estimation time: 0:28:08.300900
             Value  Std err  t-test  p-value  Rob. Std err  Rob. t-test  \
ASC_CAR      0.248    0.105    2.37   0.0177         0.111         2.24   
ASC_SM       0.917    0.122    7.53  5.2e-14         0.192         4.78   
BETA_COST    -1.77    0.103   -17.2        0         0.159        -11.1   
BETA_HE     -0.781    0.139   -5.61 2.07e-08         0.137         -5.7   
BETA_TIME     -1.7   0.0955   -17.8        0         0.207        -8.22   
CLASSIC_std   3.24    0.261    12.4        0         0.418         7.75   
RAIL_std    0.0107    0.216  0.0494    0.961        0.0319        0.335   

             Rob. p-value  
ASC_CAR            0.0249  
ASC_SM           1.71e-06  
BETA_COST               0  
BETA_HE          1.18e-08  
BETA_TIME        2.22e-16  
CLASSIC_std       9.1e-15  
RAIL_std            0.737  
Nbr of observations: 6768
LL(0) =    -5342.141
LL(beta) = -5243.217
Output file: Mixture_SM_ErrorComp_02.html


In [3]:
pandasResults

,Value,Std err,t-test,p-value,Rob. Std err,Rob. t-test,Rob. p-value
ASC_CAR,0.248,0.105,2.37,0.0177,0.111,2.24,0.0249
ASC_SM,0.917,0.122,7.53,5.2e-14,0.192,4.78,1.71e-06
BETA_COST,-1.77,0.103,-17.2,0,0.159,-11.1,0
BETA_HE,-0.781,0.139,-5.61,2.07e-08,0.137,-5.7,1.18e-08
BETA_TIME,-1.7,0.0955,-17.8,0,0.207,-8.22,2.22e-16
CLASSIC_std,3.24,0.261,12.4,0,0.418,7.75,9.1e-15
RAIL_std,0.0107,0.216,0.0494,0.961,0.0319,0.335,0.737
